### plotting layers innards
plotting T-SNE preprocessed layer similarly to first plot of
[Learning when to skim and when to read](https://metamind.io/research/learning-when-to-skim-and-when-to-read)


In [1]:
import numpy as np
import pickle
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, save
from bokeh.models import HoverTool
from bokeh.io import output_notebook
output_notebook()
from sklearn.manifold import TSNE
import pandas as pd

Loading BokehJS ...

### Load data

In [2]:
# dataframe outline
#d = {'x_dev': x_dev, 'y_dev': y_dev, 'y_net': y_net, 'prob_net': prob_net, 'layer': layer}
path_ = './misc_data/metrics.pkl'
d = pd.read_pickle(path_)
# df.to_pickle(path_)

### T-SNE 

In [3]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
tsne_repr = tsne.fit_transform(list(d['layer']))
# print (tsne_repr)


### Bokeh

In [4]:
tp_ = []  # true positive
tp_ind = []
tn_ = []  # true negative
tn_ind = []
fp_ = []  # false positive
fp_ind = []
fn_ = []  # false negative
fn_ind = []
y_dev = list(d['y_dev'])
y_net = list(d['y_net'])
for i_, val_ in enumerate(y_dev):
    if y_net[i_] == y_dev[i_]: # got it correct
        if val_ == 1:
            # positive
            tp_.append(tsne_repr[i_])
            tp_ind.append(i_)
        else:
            # negative
            tn_.append(tsne_repr[i_])
            tn_ind.append(i_)
    else:
        if val_ == 1:
            # was positive said negative
            fn_.append(tsne_repr[i_])
            fn_ind.append(i_)
        else:
            fp_.append(tsne_repr[i_])
            fp_ind.append(i_)

# print ("TP\n{}\n TN\n{}\n FP\n{}\n FN\n{}\n".format(tp_, tn_, fp_, fn_))
# print ("TP\n{}\n TN\n{}\n FP\n{}\n FN\n{}\n".format(tp_ind, tn_ind, fp_ind, fn_ind))

In [5]:
from bokeh.models import HoverTool

# 
size_ = 10
line_width_ = 2
fill_alpha_ = 0.7
# parametrize more hover @ http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#custom-tooltip
hover = HoverTool(
        tooltips=[
            #("index", "$index"),
            ("(x,y)", "(@x, @y)"),
            ("sentence", "@desc"),
            ("(neg, pos)", "@prob")
        ]
    )

TOOLS = ["pan,wheel_zoom,box_zoom,reset,save, crosshair",hover]
# (was , predicted) -- (line, filling) -- blue - > positive, red -> negative
COLORS = [('blue','blue'), ('red','red'), ('red','blue'), ('blue','red')]
LEGEND = ['true positive', 'true negative', 'false positive', 'false negative']
INDICES = [tp_ind, tn_ind, fp_ind, fn_ind]

plot = figure(title="FC network layer", tools=TOOLS,
           x_axis_location=None, y_axis_location=None)
# plot = figure(width=400, height=400)

for ind_ in range(len(INDICES)):
    if len(tsne_repr[INDICES[ind_]]) > 0:
        source = ColumnDataSource(
            data=dict(
                x=tsne_repr[INDICES[ind_]][:,0],
                y=tsne_repr[INDICES[ind_]][:,1],
                desc=np.asarray(d.get('x_dev'))[INDICES[ind_]],
                prob=np.asarray(d.get('prob_net'))[INDICES[ind_]]
            )
        )
        plot.circle(
            x='x', y='y', size=size_, source=source,
            color=COLORS[ind_][1], fill_alpha=fill_alpha_, line_width=line_width_, line_color=COLORS[ind_][0],
            legend=LEGEND[ind_])



show(plot)
# save(plot, "plot.html")

